<a href="https://colab.research.google.com/github/PedroRibeiroM/aulaJoao/blob/main/exercicioJoao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from typing_extensions import Self
import threading
import socket

class ResourceServer:
    def __init__(self, host="localhost", port=5000):
        self.host = host
        self.port = port
        self.resources = {"impressora1": True, "sala_reuniao1": True}  # True = disponível
        self.lock = threading.Lock()
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.socket.bind((self.host, self.port))
        self.socket.listen()

    def handle_request(self, client_socket, address):
        with client_socket:
            data = client_socket.recv(1024).decode()
            client_id, resource, duration = data.split(",")
            duration = int(duration)
            with self.lock:
                if self.resources[resource]:
                    self.resources[resource] = False
                    print(f"Cliente {client_id} reservou {resource} por {duration} minutos")
                    client_socket.sendall(b"True")
                    threading.Timer(duration * 60, self.release_resource, args=[client_id, resource]).start()
                else:
                    print(f"Cliente {client_id} - {resource} ocupado")
                    client_socket.sendall(b"False")

    def release_resource(self, client_id, resource):
        with self.lock:
            self.resources[resource] = True
            print(f"Cliente {client_id} liberou {resource}")

    def start_server(self):
        while True:
            client_socket, address = self.socket.accept()
            print(f"Conexão de {address} estabelecida.")
            client_handler = threading.Thread(
                target=self.handle_request, args=(client_socket, address)
            )
            client_handler.start()

class ResourceClient:
    def __init__(self, host="localhost", port=5000):
        self.host = host
        self.port = port

    def request_resource(self, client_id, resource, duration):
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.connect((self.host, self.port))
            message = f"{client_id},{resource},{duration}"
            s.sendall(message.encode())
            data = s.recv(1024)
            success = data.decode() == "True"
            if success:
                print(f"Cliente {client_id} - reserva de {resource} confirmada!")
            else:
                print(f"Cliente {client_id} - reserva de {resource} negada.")

if __name__ == "__main__":
    server = ResourceServer()
    threading.Thread(target=server.start_server).start()

    client = ResourceClient()
    client.request_resource(1, "impressora1", 10)

Conexão de ('127.0.0.1', 57678) estabelecida.
Cliente 1 reservou impressora1 por 10 minutos
Cliente 1 - reserva de impressora1 confirmada!
